# ML Wheat Yield Prediction

This code uses yearly data on temperature, rainfall, and wheat production in the top 5 wheat-producing states in Australia from 1901-2021. The data is merged and preprocessed for ML methods. FF-ANN, LSTM, and XGBoost models are constructed, trained, optimised and evaluated.

## Data preparation

#### Install libraries if necessary

In [ ]:
pip install matplotlib seaborn scikit-learn scikeras keras tensorflow lightgbm xgboost

#### Open csv files as dataframes

In [ ]:
import pandas as pd

aus_wheat = pd.read_csv('aus_wheat_yield.csv')
aus_area_planted = pd.read_csv('hectares_planted.csv')
mean_temp = pd.read_csv('aus_mean_temp.csv')
min_temp = pd.read_csv('aus_min_temp.csv')
max_temp = pd.read_csv('aus_max_temp.csv')
precipitation = pd.read_csv('aus_precipitation.csv')

#### Merge dataframes into one

In [ ]:
# List of regions
regions = ['Australia', 'New South Wales', 'Victoria', 'Queensland', 'South Australia', 'Western Australia', 'Tasmania']

# Create an empty list to store the individual DataFrames
dataframes = []

# Iterate over the regions and merge the data
for region in regions:
    region_yield_df = aus_wheat[['Year', region]].rename(columns={region: 'Wheat Yield'})
    region_area_planted_df = aus_area_planted[['Year', region]].rename(columns={region: 'Hectares Planted'})
    region_mean_temp_df = mean_temp[['Year', region]].rename(columns={region: 'Mean Temperature'})
    region_max_temp_df = max_temp[['Year', region]].rename(columns={region: 'Max Temperature'})
    region_min_temp_df = min_temp[['Year', region]].rename(columns={region: 'Min Temperature'})
    region_precipitation_df = precipitation[['Year', region]].rename(columns={region: 'Precipitation'})

    region_data = pd.merge(region_yield_df, region_mean_temp_df, on='Year')
    region_data = pd.merge(region_data, region_max_temp_df, on='Year')
    region_data = pd.merge(region_data, region_min_temp_df, on='Year')
    region_data = pd.merge(region_data, region_precipitation_df, on='Year')
    region_data = pd.merge(region_data, region_area_planted_df, on='Year')

    # Add the 'Region' column
    region_data['Region'] = region

    dataframes.append(region_data)

# Concatenate all the region DataFrames into a single DataFrame
merged_df = pd.concat(dataframes, ignore_index=True)

# Sort the DataFrame by 'Year' to have the years in order
merged_df.sort_values(by='Year', inplace=True)

# Reset the index of the DataFrame
merged_df.reset_index(drop=True, inplace=True)

# Remove unneeded regions
df_filtered = merged_df[(merged_df['Region'] != 'Australia') & (merged_df['Region'] != 'Tasmania')]

## Exploratory data analysis

#### Summary statistics

In [ ]:
# List of variables for which you want summary statistics
variables_of_interest = ['Wheat Yield', 'Mean Temperature', 'Min Temperature', 'Max Temperature', 'Precipitation', 'Hectares Planted']

# Group the data by 'Region' and calculate specific summary statistics for each variable
summary_stats = df_filtered.groupby('Region')[variables_of_interest].agg(['mean', 'std', 'min', 'max'])

# Display the summary statistics table for each variable
print("Summary Statistics for Variables by Region:")
print(summary_stats)

#### Plot wheat yield over time for all five states

In [ ]:
import matplotlib.pyplot as plt

# List of regions
regions = ['New South Wales', 'Victoria', 'Queensland', 'South Australia', 'Western Australia']

plt.figure(figsize=(10, 6))

# Iterate over the regions and plot wheat yield for each one
for region in regions:
    region_data = df_filtered[df_filtered['Region'] == region]
    plt.plot(region_data['Year'], region_data['Wheat Yield'], label=region)

plt.xlabel('Year')
plt.ylabel('Wheat Yield')
plt.title('Wheat Yield by Region over Time')
plt.legend()

plt.show()

In [ ]:
# Create a 3x2 grid of subplots
fig, axes = plt.subplots(3, 2, figsize=(12, 8))

# Flatten the axes array for easier iteration
axes = axes.flatten()

# Iterate over the regions and create plots in the grid
for i, region in enumerate(regions):
    # Filter data for the current region
    region_data = df_filtered[df_filtered['Region'] == region]

    # Plot wheat yield for the current region
    axes[i].plot(region_data['Year'], region_data['Wheat Yield'])
    axes[i].set_xlabel('Year')
    axes[i].set_ylabel('Wheat Yield')
    axes[i].set_title(f'Wheat Yield in {region}')

# Remove any empty subplots in the grid
for i in range(len(regions), len(axes)):
    fig.delaxes(axes[i])

# Adjust layout and display the plots
plt.tight_layout()
plt.show()

#### Correlation heatmap for each state

In [ ]:
import seaborn as sns
import math

# Create a 3x2 grid of subplots
num_rows = 3
num_cols = 2
fig, axes = plt.subplots(num_rows, num_cols, figsize=(16, 12))

# Flatten the axes array for easier iteration
axes = axes.flatten()

# Create correlation matrices and heatmaps for each state
for i, region in enumerate(regions):
    # Filter data for the current region
    state_data = df_filtered[df_filtered['Region'] == region]

    # Create a correlation matrix
    correlation_matrix = state_data.corr()

    # Create a heatmap in the current subplot
    sns.heatmap(correlation_matrix, ax=axes[i], annot=True, cmap='coolwarm')
    axes[i].set_title(f'Correlation Matrix for {region}')

# Remove any empty subplots in the grid
for i in range(len(regions), num_rows * num_cols):
    fig.delaxes(axes[i])

# Adjust layout and display the plots
plt.tight_layout()
plt.show()

#### Correlation heatmap for all states

In [ ]:
# Create a heatmap for the overall correlations
plt.figure(figsize=(10, 8))
sns.heatmap(df_filtered.corr(), annot=True, cmap='coolwarm')
plt.title('Correlation Matrix for All Regions')
plt.show()

#### Perform one-hot encoding to create boolean columns for each state, ready for neural networks

In [ ]:
# Perform one-hot encoding for the 'Region' column
encoded_regions = pd.get_dummies(df_filtered['Region'], prefix='Region')

# Concatenate the one-hot encoded columns with the original DataFrame
df_filtered = pd.concat([df_filtered, encoded_regions], axis=1)

# Define the inputs
inputs = ['Mean Temperature', 'Min Temperature', 'Max Temperature', 'Precipitation', 'Hectares Planted'] + list(encoded_regions.columns)

#### Center and scale data 

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# Compute the mean of each column in the data
column_means = np.mean(df_filtered[inputs], axis=0)

# Subtract the column means from each column in the data
centered_data = df_filtered[inputs] - column_means

# Create a MinMaxScaler object
scaler = MinMaxScaler()

# Fit the scaler on the centered data
scaled_data = scaler.fit_transform(centered_data)

# Assign the scaled data back to the original dataframe
df_filtered[inputs] = scaled_data

#### Save pre-processed dataframe as CSV

In [ ]:
df_filtered.to_csv('prepared_data.csv')

#### Split data into training and test sets based on the fixed split year

In [ ]:
X = df_filtered[inputs]
y = df_filtered['Wheat Yield']

# Fixed split is used as this is time series data, a random split would disrupt the temporal nature and introduce information
#leakage
split_year = 1990
X_train = df_filtered[df_filtered['Year'] < split_year][inputs]
y_train = df_filtered[df_filtered['Year'] < split_year]['Wheat Yield']

X_test = df_filtered[df_filtered['Year'] >= split_year][inputs]
y_test = df_filtered[df_filtered['Year'] >= split_year]['Wheat Yield']

## Feed-forward ANN construction

#### Train FF-ANN and evaluate on test set

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from keras.models import Sequential
from keras.regularizers import l2

# Function to calculate Mean Absolute Percentage Error (MAPE)
def mape(y_true, y_pred):
    y_true = K.cast(y_true, 'float32')
    y_pred = K.cast(y_pred, 'float32')
    diff = K.abs((y_true - y_pred) / K.clip(K.abs(y_true), K.epsilon(), None))
    return 100.0 * K.mean(diff, axis=-1)




def create_single_model():
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(X.shape[1],)))
    model.add(Dropout(0.1))
    model.add(Dense(32, activation='relu', kernel_regularizer=l2(0.01)))
    model.add(Dropout(0.1))
    model.add(Dense(8, activation='relu', kernel_regularizer=l2(0.01)))
    model.add(Dropout(0.1))
    model.add(Dense(1))  
    return model

# Create the model
ANN = create_single_model()

# Compile the model
ANN.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae', 'mape'])

# Train the model using all data
history = ANN.fit(X_train, y_train, epochs=100, batch_size=2, verbose=1)

loss, mae, mape_value = ANN.evaluate(X_test, y_test)
predictions = ANN.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, predictions))
r2 = r2_score(y_test, predictions)

#### FF-ANN Loss over epochs

In [ ]:
# Get the MSE values from the training history
train_loss = history.history['loss']

# Plot the MSE during training
plt.plot(train_loss)
plt.title('Training MSE')
plt.xlabel('Epoch')
plt.ylabel('MSE')
plt.show()

#### FF-ANN evaluation on test set

In [ ]:
#Print evaluation metrics
print('MSE (FF-ANN) = ', loss)
print('MAE (FF-ANN) = ', mae)
print('RMSE (FF-ANN) = ', rmse)
print('MAPE (FF-ANN) = ', mape_value)
print('R2 (FF-ANN) = ', r2)

#### FF-ANN state metrics

In [ ]:
# Iterate over the regions and evaluate the single model for each region
for region in regions:
    region_data = df_filtered[df_filtered['Region'] == region]
    X_test_region = region_data[region_data['Year'] >= split_year][inputs]
    y_test_region = region_data[region_data['Year'] >= split_year]['Wheat Yield']

    # Make predictions for the region using the single model
    predictions_region = ANN.predict(X_test_region)

    # Calculate evaluation metrics for the region
    mse_region = mean_squared_error(y_test_region, predictions_region)
    mae_region = mean_absolute_error(y_test_region, predictions_region)
    rmse_region = np.sqrt(mse_region)
    r2_region = r2_score(y_test_region, predictions_region)
    mape_region = mape(y_test_region, predictions_region)

    # Display the evaluation results for the region
    print(f"\nResults for {region}:")
    print("MSE:", mse_region)
    print("MAE:", mae_region)
    print("RMSE:", rmse_region)
    print("R2 Score:", r2_region)
    print("MAPE:", np.mean(mape_region))


#### FF-ANN predicted vs actual test set values

In [ ]:
# Create a 3x2 grid of subplots for the predicted vs. actual yield plots
fig, axes = plt.subplots(3, 2, figsize=(12, 8))
fig.suptitle('Predicted vs. Actual Wheat Yield for Each State (Feedforward ANN)', fontsize=16)

# Flatten the axes array for easier iteration
axes = axes.flatten()

# Iterate over the regions and create plots in the grid
for i, region in enumerate(regions):
    region_data = df_filtered[df_filtered['Region'] == region]
    X_test_region = region_data[region_data['Year'] >= split_year][inputs]
    y_test_region = region_data[region_data['Year'] >= split_year]['Wheat Yield']
    predictions_region = ANN.predict(X_test_region)

    # Get the years in the test set for this region
    years_test_region = region_data[region_data['Year'] >= split_year]['Year']

    # Plot the actual yield as a blue line
    axes[i].plot(years_test_region, y_test_region, label='Actual Yield', color='blue')
    
    # Plot the predicted yield as an orange line
    axes[i].plot(years_test_region, predictions_region, label='Predicted Yield', color='orange')
    
    axes[i].set_xlabel('Year')
    axes[i].set_ylabel('Yield')
    axes[i].set_title(region)
    axes[i].legend()

# Remove any empty subplots in the grid
for i in range(len(regions), len(axes)):
    fig.delaxes(axes[i])

# Adjust layout and display the plots
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()


## LSTM construction

#### LSTM training

In [ ]:
from keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.layers import LSTM

# Define generator
n_input = 4
n_features = X_train.shape[1] 

generator = TimeseriesGenerator(X_train.values, y_train.values, length=n_input, batch_size=4)

# Continue with model definition and training
LSTM_model = Sequential()
LSTM_model.add(LSTM(32, kernel_regularizer=l2(0.05), return_sequences=True, input_shape=(n_input, n_features)))
LSTM_model.add(Dropout(0.1))
LSTM_model.add(LSTM(8, kernel_regularizer=l2(0.05)))
LSTM_model.add(Dropout(0.1))
LSTM_model.add(Dense(1))
LSTM_model.compile(loss='mean_squared_error', optimizer= 'adam')
LSTM_model.summary()


LSTM_model.fit(generator, epochs=100)

#### LSTM Loss over epochs

In [ ]:
loss_per_epoch = LSTM_model.history.history['loss']
plt.plot(range(len(loss_per_epoch)), loss_per_epoch)
plt.title('Training MSE (LSTM)')
plt.xlabel('Epoch')
plt.ylabel('MSE')
plt.show()

#### LSTM evaluation on test set

In [ ]:
# For evaluation on the test data
test_generator = TimeseriesGenerator(X_test.values, y_test.values, length=n_input, batch_size=1)
y_pred_test = LSTM_model.predict(test_generator)

# Get the corresponding years for the test set
years_test_lstm = df_filtered[df_filtered['Year'] >= split_year]['Year'][n_input:].values

# Evaluate the model on the test set
test_mse = mean_squared_error(y_test.values[n_input:], y_pred_test)
test_mae = mean_absolute_error(y_test.values[n_input:], y_pred_test)
r2 = r2_score(y_test.values[n_input:], y_pred_test)

# Calculate RMSE and MAPE using existing functions
test_rmse = np.sqrt(test_mse)
test_mape = mape(y_test.values[n_input:], y_pred_test)

print("LSTM MSE:", test_mse)
print("LSTM MAE:", test_mae)
print("LSTM R-squared:", r2)
print("LSTM RMSE:", test_rmse)
print("LSTM MAPE:", np.mean(test_mape))

#### LSTM predicted vs actual test set values 

In [ ]:
# Create a 3x2 grid of subplots for the predicted vs. actual yield plots
fig, axes = plt.subplots(3, 2, figsize=(12, 8))
fig.suptitle('Predicted vs. Actual Wheat Yield for Each State (LSTM)', fontsize=16)

# Flatten the axes array for easier iteration
axes = axes.flatten()

# Iterate over the regions and create plots in the grid
for i, region in enumerate(regions):
    region_data = df_filtered[df_filtered['Region'] == region]
    X_test_region = region_data[region_data['Year'] >= split_year][inputs]
    y_test_region = region_data[region_data['Year'] >= split_year]['Wheat Yield']
    
    # Generate sequences for LSTM predictions
    lstm_test_generator = TimeseriesGenerator(X_test_region.values, y_test_region.values, length=n_input, batch_size=1)
    lstm_predictions_region = LSTM_model.predict(lstm_test_generator)
    
    # Get the years in the test set for this region
    years_test_region = region_data[region_data['Year'] >= split_year]['Year']
    

    # Plot the actual yield as a blue line
    axes[i].plot(years_test_region, y_test_region, label='Actual Yield', color='blue')
    
    # Plot the predicted yield as an orange line (LSTM)
    axes[i].plot(years_test_region.values[n_input:], lstm_predictions_region, label='LSTM Predicted Yield', color='orange')

    
    axes[i].set_xlabel('Year')
    axes[i].set_ylabel('Yield')
    axes[i].set_title(region)
    axes[i].legend()

# Remove any empty subplots in the grid
for i in range(len(regions), len(axes)):
    fig.delaxes(axes[i])

# Adjust layout and display the plots
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()


#### LSTM state metrics

In [ ]:
# Create a dictionary to store state-wise results
state_results_lstm = {}

# Iterate over the regions and evaluate the LSTM model for each region
for region in regions:
    state_data = df_filtered[df_filtered['Region_' + region] == 1]
    X_state = state_data[inputs]
    y_state = state_data['Wheat Yield']

    # Create a separate generator for predictions on the test set
    state_test_generator_lstm = TimeseriesGenerator(X_state.values, y_state.values, length=n_input, batch_size=1)
    y_pred_state_lstm = LSTM_model.predict(state_test_generator_lstm)

    # Evaluate the model for the region on the test set
    state_test_mse_lstm = mean_squared_error(y_state.values[n_input:], y_pred_state_lstm)
    state_test_mae_lstm = mean_absolute_error(y_state.values[n_input:], y_pred_state_lstm)
    state_r2_lstm = r2_score(y_state.values[n_input:], y_pred_state_lstm)
    
    # Calculate RMSE and MAPE using existing functions
    state_test_rmse_lstm = np.sqrt(state_test_mse_lstm)
    state_test_mape_lstm = mape(y_state.values[n_input:], y_pred_state_lstm)

    # Store the results for the region
    state_results_lstm[region] = {
        'Test MSE': state_test_mse_lstm,
        'Test MAE': state_test_mae_lstm,
        'Test RMSE': state_test_rmse_lstm,
        'Test MAPE': np.mean(state_test_mape_lstm),
        'R-squared': state_r2_lstm
    }

print("State-wise Model Performance - LSTM:")
for region, results_lstm in state_results_lstm.items():
    print(f"Region: {region}")
    print("LSTM MSE:", results_lstm['Test MSE'])
    print("LSTM MAE:", results_lstm['Test MAE'])
    print("LSTM RMSE:", results_lstm['Test RMSE'])
    print("LSTM MAPE:", results_lstm['Test MAPE'])
    print("LSTM R-squared:", results_lstm['R-squared'])
    print()


## XGBoost construction

#### Grid search for optimal hyper-parameters

In [ ]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

# Define parameters for the XGBoost model
xgb_params = {
    'objective': 'reg:squarederror', 
    'eval_metric': 'rmse',            
}

# Define the dataset
dtrain = xgb.DMatrix(X_train, label=y_train)

# Define the parameter grid for the grid search
param_grid = {
    'learning_rate': [0.3, 0.6],       
    'max_depth': [4, 6],                 
    'min_child_weight': [1, 3],          
    'subsample': [0.8, 1.0],                
    'n_estimators': [50, 100],
    'gamma': [0, 0.2],
    'reg_alpha': [0, 0.1, 1],
    'reg_lambda': [0, 0.1, 1],
}

# Create an XGBoost regressor
xgb_model = xgb.XGBRegressor(**xgb_params)

# Perform grid search
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=3)
grid_search.fit(X_train, y_train)

# Get the best parameters from the grid search
best_params_xgb = grid_search.best_params_
print("Best Parameters for XGBoost:", best_params_xgb)

#### XGBoost training and testing 

In [ ]:
# Train the XGBoost model with the best parameters
best_xgb_model = xgb.XGBRegressor(**xgb_params, **best_params_xgb)
best_xgb_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_test_xgb = best_xgb_model.predict(X_test)

# Get the corresponding years for the test set
years_test_xgb = df_filtered[df_filtered['Year'] >= split_year]['Year'].values

# Evaluate the model on the test set
test_mse_xgb = mean_squared_error(y_test, y_pred_test_xgb)
test_mae_xgb = mean_absolute_error(y_test, y_pred_test_xgb)
r2_xgb = r2_score(y_test, y_pred_test_xgb)

# Calculate additional metrics
test_rmse_xgb = np.sqrt(test_mse_xgb)
test_mape_xgb = np.mean(mape(y_test, y_pred_test_xgb))

print("Test MSE (XGBoost):", test_mse_xgb)
print("Test MAE (XGBoost):", test_mae_xgb)
print("Test R-squared (XGBoost):", r2_xgb)
print("Test RMSE (XGBoost):", test_rmse_xgb)
print("Test MAPE (XGBoost):", test_mape_xgb)

#### XGBoost predicted vs actual test set values

In [ ]:
# Create a DataFrame to store the results for XGBoost
results_df_xgb = pd.DataFrame({
    'Year': years_test_xgb,
    'Actual_Yield': y_test.values,
    'Predicted_Yield': y_pred_test_xgb
})

# Create a 3x2 grid of subplots for the predicted vs. actual yield plots for XGBoost
fig, axes = plt.subplots(3, 2, figsize=(12, 8))
fig.suptitle('Predicted vs. Actual Wheat Yield for Each State (XGBoost)', fontsize=16)

# Flatten the axes array for easier iteration
axes = axes.flatten()

# Iterate over the regions and create plots in the grid for XGBoost
for i, region in enumerate(regions):
    region_data = df_filtered[df_filtered['Region'] == region]
    X_test_region = region_data[region_data['Year'] >= split_year][inputs]
    y_test_region = region_data[region_data['Year'] >= split_year]['Wheat Yield']
    
    # Predictions for XGBoost
    y_pred_region_xgb = best_xgb_model.predict(X_test_region)
    
    # Get the years in the test set for this region
    years_test_region = region_data[region_data['Year'] >= split_year]['Year']
    
    # Plot the actual yield as a blue line
    axes[i].plot(years_test_region, y_test_region, label='Actual Yield', color='blue')
    
    # Plot the predicted yield as a red line (XGBoost)
    axes[i].plot(years_test_region.values, y_pred_region_xgb, label='XGBoost Predicted Yield', color='orange')

    axes[i].set_xlabel('Year')
    axes[i].set_ylabel('Yield')
    axes[i].set_title(region)
    axes[i].legend()

# Remove any empty subplots in the grid for XGBoost
for i in range(len(regions), len(axes)):
    fig.delaxes(axes[i])

# Adjust layout and display the plots for XGBoost
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

#### XGBoost state metrics

In [ ]:
# Create a dictionary to store state-wise results for XGBoost
state_results_xgb = {}

# Iterate over the regions and evaluate the XGBoost model for each region
for region in regions:
    state_data = df_filtered[df_filtered['Region_' + region] == 1]
    X_state = state_data[inputs]
    y_state = state_data['Wheat Yield']

    # Predictions for XGBoost
    y_pred_state_xgb = best_xgb_model.predict(X_state)

    # Evaluate the model for the region on the test set
    state_test_mse_xgb = mean_squared_error(y_state, y_pred_state_xgb)
    state_test_mae_xgb = mean_absolute_error(y_state, y_pred_state_xgb)
    state_r2_xgb = r2_score(y_state, y_pred_state_xgb)
    state_rmse_xgb = np.sqrt(state_test_mse_xgb)
    state_mape_xgb = np.mean(mape(y_state, y_pred_state_xgb))

    # Store the results for the region
    state_results_xgb[region] = {
        'Test MSE (XGBoost)': state_test_mse_xgb,
        'Test MAE (XGBoost)': state_test_mae_xgb,
        'R-squared (XGBoost)': state_r2_xgb,
        'Test RMSE (XGBoost)': state_rmse_xgb,
        'Test MAPE (XGBoost)': state_mape_xgb,
    }

# Output state-wise results for XGBoost
print("State-wise Model Performance - XGBoost:")
for region, results_xgb in state_results_xgb.items():
    print(f"Region: {region}")
    print("Test MSE (XGBoost):", results_xgb['Test MSE (XGBoost)'])
    print("Test MAE (XGBoost):", results_xgb['Test MAE (XGBoost)'])
    print("R-squared (XGBoost):", results_xgb['R-squared (XGBoost)'])
    print("Test RMSE (XGBoost):", results_xgb['Test RMSE (XGBoost)'])
    print("Test MAPE (XGBoost):", results_xgb['Test MAPE (XGBoost)'])
    print()

#### Show relative feature importance for environmental features

In [ ]:
# Display feature importance
feature_importance = best_xgb_model.feature_importances_
feature_names = X_train.columns

# Create a DataFrame to store feature importance
feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importance})

# Sort features by importance in descending order
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Define the environmental features you want to show
selected_features = ["Hectares Planted", "Precipitation", "Min Temperature", "Max Temperature", "Mean Temperature"]

# Filter feature importance DataFrame
filtered_feature_importance_df = feature_importance_df[
    feature_importance_df['Feature'].isin(selected_features)
]

# Sort features by importance in descending order
filtered_feature_importance_df = filtered_feature_importance_df.sort_values(by='Importance', ascending=False)

# Plot filtered feature importance
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=filtered_feature_importance_df, palette='viridis')
plt.title('XGBoost Feature Importance (Selected Environmental Features)')
plt.show()